### Description
This notebook contains all code used in the creation of the MassBay_2017_to_2022.csv file.

# MWRA/MassBay Data Concatenation

In [68]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import dateparser
from pytz import timezone
import shutil
import datetime

In [69]:
data_dir = '../../data/'

## OldFormat Concatenation (2017 - 2019)

In [70]:
old_data_dir = data_dir + 'MassBay/OldFormat/'

#### Load old format excel files

In [71]:
whitelist = [ # Files that have more than one sheet but only first sheet is significant
  'WN172 event dataMarch2017.xlsx',
 'WN173 sensor dataApril2017.xlsx',
 'WN177 Event dataAug2017.xlsx']
 
# check that each excel file in the data_dir has only one sheet, add to blacklist if not
csvs = {}
for file in os.listdir(old_data_dir):
    if file.endswith('.xlsx'):
        all_sheets = pd.read_excel(old_data_dir + file, sheet_name=None)
        if len(all_sheets) > 1 and file not in whitelist:
          print('ERROR:', file, 'has', len(all_sheets), 'sheets')
          continue
        else:
          first_sheet = all_sheets[list(all_sheets.keys())[0]]
          csvs[file] = first_sheet

#### Correct column inconsistencies

In [72]:
# Correct insignificant column inconsistency in June 2018 data
june2018 = 'WN185 Event dataJune2018.xlsx'
cols_to_drop = []
columns = set(list(csvs.values())[0].columns)
for col in csvs[june2018].columns:
    if col not in columns:
        cols_to_drop.append(col)
csvs[june2018].drop(columns = cols_to_drop, inplace = True)

# Correct inisgnificant column inconsistency in Feb 2017 data
csvs['WN171 Event dataFeb2017.xlsx'].drop(columns='Altitude', inplace=True)

# check that all elements of csvs have the same columns
columns = set(list(csvs.values())[0].columns)
print(f'Columns in common: {columns}')
for item in csvs.items():
  if columns != set(item[1].columns):
        print("ERROR: Column inconsistency in :" +item[0]) # error  if not all elements of csvs have the same columns (should be the same)
        break

Columns in common: {'Longitude', 'Chla Fluor', 'DO % Saturation', 'SampleDateTime', 'Temperature', 'Salinity', 'pH <2>', 'SampleID', 'Sigma-T', 'Air Irradiance', 'Depth', 'StationID', 'Dissolved Oxygen (Model 43)', 'Water Irradiance', 'Beam Attenuation', 'Latitude', 'Conductivity'}


#### Concatenate the files

In [73]:
# concatenate all csvs into one dataframe and save as csv
csv_list = list(csvs.values())
old_format_concat = pd.concat(csv_list, ignore_index=True, sort=False)

#### Post-process concatenated dataframe

In [74]:
#add data source column
old_format_concat['Data Source'] = 'MassBay_2017_to_2019_CONCAT'

#### Export as csv

In [75]:
# Save as csv
old_format_concat.to_csv(data_dir + '/MassBay/OldFormat/concat/MassBay_2017_to_2019_CONCAT.csv', index = False)

## New Format Concatenation (2020-)

In [76]:
new_data_dir = data_dir + 'MassBay/NewFormat/'

#### Load new format excel files

In [77]:
whitelist = []
# check that each excel file in the data_dir has only one sheet, add to blacklist if not
newcsvs = {}
for file in os.listdir(new_data_dir):
    if file.endswith('.xlsx'):
        all_sheets = pd.read_excel(new_data_dir + file, sheet_name=None)
        if len(all_sheets) != 1 and file not in whitelist:
            print('ERROR:', file, 'has', len(all_sheets), 'sheets')
        else:
            first_sheet = all_sheets[list(all_sheets.keys())[0]]
            newcsvs[file] = first_sheet

#### Correct column inconsistencies

In [78]:
# Correct column mapping inconsistencies in new format
# Drop inconsistent columns
for csv in newcsvs:
    if 'DO_RAW (mg/L)' in newcsvs[csv].columns:
        newcsvs[csv]['DISS_OXYGEN (mg/L)'] = newcsvs[csv]['DO_RAW (mg/L)']
        newcsvs[csv].drop(columns = ['DO_RAW (mg/L)'], inplace = True)
    if 'PCT_SAT_RAW (PCT)' in newcsvs[csv].columns:
        newcsvs[csv]['PCT_SAT (PCT)'] = newcsvs[csv]['PCT_SAT_RAW (PCT)']
        newcsvs[csv].drop(columns = ['PCT_SAT_RAW (PCT)'], inplace = True)
        
# Remove extraneous columns in March 2022 data
newcsvs["WN222_Event_dataMar2022.xlsx"].drop(columns = ['Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35'], inplace = True)

#### Concatenate files

In [79]:
# Concatenate
newconcat = pd.concat(newcsvs, ignore_index=True, sort=False)

#### Post-process concatenated dataframe

In [80]:
# Remove exraneous colulms from concat
newconcat = newconcat.drop(columns = ['COMMENTS'])

# Add data source column
newconcat['Data Source'] = 'MassBay_2020_to_2022_CONCAT'

# Remove space from station col
newconcat['STAT_ID'] = [stat.strip(" ") for stat in newconcat['STAT_ID']]

# Convert timestamp to datetime if string
for ind, row in newconcat.iterrows():
    date_str = row['PROF_DATE_TIME_LOCAL']
    if type(date_str) == str:
        newconcat.loc[ind, 'PROF_DATE_TIME_LOCAL'] = dateparser.parse(date_str)

len(newconcat) == 1257

/Users/becklabash/opt/anaconda3/lib/python3.7/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


True

#### Export as csv

In [81]:
# Save concat
newconcat.to_csv(data_dir + 'MassBay/NewFormat/concat/MassBay_2020_to_2022_CONCAT.csv', index=False)

## Merging Old and New Formats

#### Load the old and new format concatenations

In [82]:
old_format = pd.read_csv(data_dir + 'MassBay/OldFormat/concat/MassBay_2017_to_2019_CONCAT.csv')
new_format = pd.read_csv(data_dir + 'MassBay/NewFormat/concat/MassBay_2020_to_2022_CONCAT.csv')

#### Map the new colums to the old columns for concatenation

In [83]:
# New:Old Column Mapping (retain new column names)
new_to_old = {
  'EVENT_ID' : 'SampleID',
 'STAT_ID' : 'StationID',
 'PROF_DATE_TIME_LOCAL' : 'SampleDateTime',
 'LATITUDE' : 'Latitude',
 'LONGITUDE' : 'Longitude',
 'DEPTH (m)': 'Depth',
 'CONDTVY (mS/cm)': 'Conductivity',
'DISS_OXYGEN (mg/L)' : 'Dissolved Oxygen (Model 43)',
'FLU_RAW (ug/L)' : 'Chla Fluor',
'PCT_SAT (PCT)' : 'DO % Saturation',
'pH ()': 'pH <2>',
 'SAL (PSU)' : 'Salinity',
'SIGMA_T ()': 'Sigma-T',
 'TEMP (C)' : 'Temperature',
'TRANS (m-1)' : 'Beam Attenuation'}

In [84]:
old_converted = old_format.copy()
for new_col in new_to_old:
    old_converted[new_col] = old_converted[new_to_old[new_col]]
    old_converted = old_converted.drop(columns = [new_to_old[new_col]])

#### Concatenate the formats

In [85]:
oldAndNew = pd.concat([new_format, old_converted], ignore_index=True, sort=False)

#### Post-process the concatenated dataframe

In [86]:
oldAndNew['Data Source'] = 'MassBay_2017_to_2022'

In [87]:
# Check dates correct
for ind, row in oldAndNew.iterrows():
    date = dateparser.parse(row['PROF_DATE_TIME_LOCAL'])
    try:
        x = date.strftime("%Y")
        if not x in ["2017", "2018", "2019", "2020", "2021", "2022"]:
            print(date_str)
    except:
        print(date_str)
    oldAndNew.loc[ind, 'PROF_DATE_TIME_LOCAL'] = date

In [88]:
eastern = timezone("US/Eastern")
# Drop microseconds and set timezone
oldAndNew['PROF_DATE_TIME_LOCAL'] = [date.replace(microsecond=0).replace(tzinfo=eastern) for date in oldAndNew['PROF_DATE_TIME_LOCAL']]

#Convert to excel parsable string
oldAndNew['PROF_DATE_TIME_LOCAL'] = [date.strftime("%Y-%m-%d %H:%M:%S%z") for date in oldAndNew['PROF_DATE_TIME_LOCAL']]

#### Export as csv

In [89]:
oldAndNew.to_csv(data_dir + '/MassBay/concat/MassBay_2017_to_2022.csv', index = False)

In [90]:
def print_column_descriptions(df):
    ind = 1
    for col in df.columns:
        print(f'Column {ind}: {col} is COLUMN DESCRIPTION   BAD FLAG : -1.E+34')
        ind += 1